In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.figure_factory as ff
import plotly.graph_objects as go

In [2]:
# Read in the data
overtime_hours = pd.read_csv('../data/csv/work_hours_overtime.csv', index_col='date', parse_dates=True)
base_hours = pd.read_csv('../data/csv/work_hours_base.csv', index_col='date', parse_dates=True)
portions = pd.read_csv('../data/csv/portions.csv', index_col='date', parse_dates=True)
employees = pd.read_csv('../data/csv/employees.csv', index_col='employee', dtype={'employee': str})

# Set the column names
overtime_hours.columns.name='employee'
base_hours.columns.name='employee'

# Calculate total work hours
total_hours = overtime_hours + base_hours
total_hours.head()

employee,4000175,4000182,4000220,4000239,4000240,4000241,4000243,4000244,4000245,4000246,...,9753767,9784946,9795581,9798848,9814343,9869495,9893478,9926881,9965178,9972531
date,,,,,,,,,,,,,,,,,,,,,
2022-07-23,525,732,585,627,732,627,732,627,480,732,...,633,640,898,530,480,908,480,1257,750,605
2022-07-24,1245,585,732,732,480,732,732,627,480,732,...,604,641,495,480,532,480,604,480,601,607
2022-07-25,480,732,732,732,732,732,627,480,480,732,...,495,642,480,994,536,894,605,531,599,603
2022-07-26,525,585,732,627,732,627,480,627,480,732,...,745,492,902,480,850,598,603,528,589,603
2022-07-27,480,732,732,732,627,627,732,480,480,732,...,494,642,494,531,895,911,747,480,602,589


In [3]:
# Splitting the job titles into different categories
food_related = [
    'آشپز 1',
    'آشپز 2',
    'سر آشپز',
    'سر شیفت',
    'سر شیفت آماده سازی',
    'سر شیفت برنج',
    'سردخانه دار',
    'سرپرست تولید',
    'قصاب',
    'کمک آشپز',
    'کمک انباردار',
    'انباردار',
    'اپراتور انبار',
]

distribution_related = [
    'سرگارسون',
    'گارسون',
    'کارگر رستوران',
    'کارگر ساده',
]

hr_related = [
    'تعمیرکار',
    'متصدی اداری',
    'متصدی برنامه ریزی و سفارشات',
    'مدیرپروژه',
    'مسئول امور اداری و منابع انسانی',
    'کارشناس',
    'کارشناس سلامت ایمنی محیط',
    'کارمند اداری',
]

In [4]:
# Function to split the given tables based on above categories
def split_table(table):
    """
    Splits the table into three different tables based on the job title
    """
    total_daily_portions = portions.sum(axis=1) # Daily portion demand

    # Group the table by job title
    grouped_work_hrs = table.transpose().groupby(employees['job_title'])
    grouped_daily_work_hrs = grouped_work_hrs.sum() # Total work hours per day per job group
    daily_employee_count = grouped_work_hrs.sum().sum() # Total number of working employees per day

    # Extract food & distribution related work hours
    food_work_hours = grouped_daily_work_hrs.loc[food_related] # food related
    dist_work_hours = grouped_daily_work_hrs.loc[distribution_related] # distribution related

    # Divide the total daily work hours by the total daily portions for each job category
    food_time_cost = food_work_hours.divide(total_daily_portions, axis=1).transpose()
    distribution_time_cost = dist_work_hours.divide(total_daily_portions, axis=1).transpose()

    # Calculate HR related time cost (total work hours of hr related employees / total number of employees)
    hr_work_hours = grouped_daily_work_hrs.loc[hr_related] # hr related
    hr_time_cost = hr_work_hours.divide(daily_employee_count, axis=1).transpose()

    return food_time_cost, distribution_time_cost, hr_time_cost

In [5]:
# Split the tables for total and overtime costs
food_time_cost, distribution_time_cost, hr_time_cost = split_table(total_hours)
food_overtime_cost, distribution_overtime_cost, hr_overtime_cost = split_table(overtime_hours)

In [6]:
# Plot the time cost tables
fig = go.Figure()
fig.add_trace(go.Scatter(x=hr_time_cost.index, y=hr_time_cost.sum(axis=1), name='HR'))
fig.add_trace(go.Scatter(x=distribution_time_cost.index, y=distribution_time_cost.sum(axis=1), name='Distribution'))
fig.add_trace(go.Scatter(x=food_time_cost.index, y=food_time_cost.sum(axis=1), name='Food'))
fig.update_layout(
    title='Time Cost',
    xaxis_title='Date',
    yaxis_title='Time Cost (minutes/portion(employee for HR))',
    legend_title='Job Category',
)
fig.show()

# export to png
# fig.write_image("../figures/time_cost_graph.png", width=1500, height=600)

In [7]:
# Plot the overtime cost tables
fig = go.Figure()
fig.add_trace(go.Scatter(x=hr_overtime_cost.index, y=hr_overtime_cost.sum(axis=1), name='HR'))
fig.add_trace(go.Scatter(x=distribution_overtime_cost.index, y=distribution_overtime_cost.sum(axis=1), name='Distribution'))
fig.add_trace(go.Scatter(x=food_overtime_cost.index, y=food_overtime_cost.sum(axis=1), name='Food'))
fig.update_layout(
    title='Overtime Cost',
    xaxis_title='Date',
    yaxis_title='Overtime Cost (minutes/portion(employee for HR))',
    legend_title='Job Category',
)
fig.show()

# export to png
# fig.write_image("../figures/overtime_cost_graph.png", width=1500, height=600)

In [16]:
import pyomo.environ as pyo
import datetime

In [48]:
# Create a model
model = pyo.ConcreteModel()

# --------------------------- INDICES ------------------------------
# Set of days and employees
model.DAYS = pyo.Set(initialize=total_hours.index, ordered=True)
model.CEMP = pyo.Set(initialize=total_hours.columns, ordered=False)

# Set of jobs per category
model.FG = pyo.Set(initialize=food_related, ordered=False) # food related
model.DG = pyo.Set(initialize=distribution_related, ordered=False) # distribution related
model.HG = pyo.Set(initialize=hr_related, ordered=False) # hr related
model.TG = model.FG.union(model.DG).union(model.HG) # all job groups
# ------------------------------------------------------------------

# --------------------------- PARAMETERS ---------------------------
# Base Wages (Calculated ver 23 work days)
model.add_component('CE_WAGES', pyo.Param(model.CEMP,initialize=employees.loc[model.CEMP, 'wage']/(184*60) )) # current employees' wages
model.add_component('FG_WAGES', pyo.Param(model.FG, initialize=employees.groupby('job_title').mean().loc[model.FG, 'wage']/(184*60))) # food related wages
model.add_component('DG_WAGES', pyo.Param(model.DG, initialize=employees.groupby('job_title').mean().loc[model.DG, 'wage']/(184*60))) # distribution related wages
model.add_component('HG_WAGES', pyo.Param(model.HG, initialize=employees.groupby('job_title').mean().loc[model.HG, 'wage']/(184*60))) # hr related wages

# Overtime Wages (1.4 Base Wages)
model.add_component('CE_OT_WAGES', pyo.Param(model.CEMP,initialize=employees.loc[model.CEMP, 'wage']/(184*60)*1.4 )) # current employees' overtime wages
model.add_component('FG_OT_WAGES', pyo.Param(model.FG, initialize=employees.groupby('job_title').mean().loc[model.FG, 'wage']/(184*60)*1.4)) # food related overtime wages
model.add_component('DG_OT_WAGES', pyo.Param(model.DG, initialize=employees.groupby('job_title').mean().loc[model.DG, 'wage']/(184*60)*1.4)) # distribution related overtime wages
model.add_component('HG_OT_WAGES', pyo.Param(model.HG, initialize=employees.groupby('job_title').mean().loc[model.HG, 'wage']/(184*60)*1.4)) # hr related overtime wages

# Time Costs
def food_cost_rule(model, day, fg):
    return food_time_cost.loc[str(day), fg]
model.FG_COST = pyo.Param(model.DAYS, model.FG, initialize=food_cost_rule)

def distribution_cost_rule(model, day, dg):
    return distribution_time_cost.loc[str(day), dg]
model.DG_COST = pyo.Param(model.DAYS, model.DG, initialize=distribution_cost_rule)

def hr_cost_rule(model, day, hg):
    return hr_time_cost.loc[str(day), hg]
model.HG_COST = pyo.Param(model.DAYS, model.HG, initialize=hr_cost_rule)
# ----------------------------------------------------------------

# -------------------------  VARIABLES ---------------------------
model.add_component('CE_BASE_VAR', pyo.Var(model.DAYS, model.CEMP, domain=pyo.NonNegativeReals))
model.add_component('CE_OT_VAR', pyo.Var(model.DAYS, model.CEMP, domain=pyo.NonNegativeReals))
model.add_component('RC_FG_BASE_VAR', pyo.Var(model.DAYS, model.FG, domain=pyo.NonNegativeReals))
model.add_component('RC_DG_BASE_VAR', pyo.Var(model.DAYS, model.DG, domain=pyo.NonNegativeReals))
model.add_component('RC_HG_BASE_VAR', pyo.Var(model.DAYS, model.HG, domain=pyo.NonNegativeReals))
model.add_component('RC_FG_OT_VAR', pyo.Var(model.DAYS, model.FG, domain=pyo.NonNegativeReals))
model.add_component('RC_DG_OT_VAR', pyo.Var(model.DAYS, model.DG, domain=pyo.NonNegativeReals))
model.add_component('RC_HG_OT_VAR', pyo.Var(model.DAYS, model.HG, domain=pyo.NonNegativeReals))
# ----------------------------------------------------------------

# -------------------------  OBJECTIVE ---------------------------
def obj_rule(model):
    result =  sum(model.CE_WAGES[ce]*model.CE_BASE_VAR[day, ce] for day in model.DAYS for ce in model.CEMP)
    result += sum(model.CE_OT_WAGES[ce]*model.CE_OT_VAR[day, ce] for day in model.DAYS for ce in model.CEMP)
    result += sum(model.FG_WAGES[fg]*model.RC_FG_BASE_VAR[day, fg] for day in model.DAYS for fg in model.FG)
    result += sum(model.DG_WAGES[dg]*model.RC_DG_BASE_VAR[day, dg] for day in model.DAYS for dg in model.DG)
    result += sum(model.HG_WAGES[hg]*model.RC_HG_BASE_VAR[day, hg] for day in model.DAYS for hg in model.HG)
    result += sum(model.FG_OT_WAGES[fg]*model.RC_FG_OT_VAR[day, fg] for day in model.DAYS for fg in model.FG)
    result += sum(model.DG_OT_WAGES[dg]*model.RC_DG_OT_VAR[day, dg] for day in model.DAYS for dg in model.DG)
    result += sum(model.HG_OT_WAGES[hg]*model.RC_HG_OT_VAR[day, hg] for day in model.DAYS for hg in model.HG)
    return result
model.add_component('OBJ', pyo.Objective(rule=obj_rule, sense=pyo.minimize))
# ----------------------------------------------------------------

# ------------------------- CONSTRAINTS --------------------------
def get_base_hours(day):
    if day.weekday() in [3,4]:
        return 0
    else:
        return 8*60

def get_daily_demand(day, job_group):
    return employees[employees['job_group'] == job_group].index

def ce_base_rule(model, day, ce):
    return model.CE_BASE_VAR[day, ce] == get_base_hours(day)
model.add_component('CE_BASE_CONST', pyo.Constraint(model.DAYS, model.CEMP, rule=ce_base_rule))

def fg_base_rule(model, day, fg):
    if day.weekday() in [3,4]:
        return model.RC_FG_BASE_VAR[day, fg] == 0
    else:
        return model.RC_FG_BASE_VAR[day, fg] >= 0
model.add_component('FG_BASE_CONST', pyo.Constraint(model.DAYS, model.FG, rule=fg_base_rule))

def dg_base_rule(model, day, dg):
    if day.weekday() in [3,4]:
        return model.RC_DG_BASE_VAR[day, dg] == 0
    else:
        return model.RC_DG_BASE_VAR[day, dg] >= 0
model.add_component('DG_BASE_CONST', pyo.Constraint(model.DAYS, model.DG, rule=dg_base_rule))

def hg_base_rule(model, day, hg):
    if day.weekday() in [3,4]:
        return model.RC_HG_BASE_VAR[day, hg] == 0
    else:
        return model.RC_HG_BASE_VAR[day, hg] >= 0
model.add_component('HG_BASE_CONST', pyo.Constraint(model.DAYS, model.HG, rule=hg_base_rule))

def daily_demand_rule(model, day, jg):
    demand = 0
    current_employees = employees.loc[employees['job_title'] == jg].index
    left_side = sum(model.CE_BASE_VAR[day, ce] for ce in current_employees)
    left_side += sum(model.CE_OT_VAR[day, ce] for ce in current_employees)
    if jg in food_related:
        demand = portions.sum(axis=1).loc[str(day)]*food_time_cost.loc[str(day), jg]
        left_side += model.RC_FG_BASE_VAR[day, jg]
        left_side += model.RC_FG_OT_VAR[day, jg]
    if jg in distribution_related:
        demand = portions.sum(axis=1).loc[str(day)]*distribution_time_cost.loc[str(day), jg]
        left_side += model.RC_DG_BASE_VAR[day, jg]
        left_side += model.RC_DG_OT_VAR[day, jg]
    if jg in hr_related:
        vars =  sum(model.CE_BASE_VAR[day, ce] for ce in model.CEMP)
        vars += sum(model.CE_OT_VAR[day, ce] for ce in model.CEMP)
        vars += sum(model.RC_FG_BASE_VAR[day, fg] for fg in model.FG)
        vars += sum(model.RC_DG_BASE_VAR[day, dg] for dg in model.DG)
        vars += sum(model.RC_HG_BASE_VAR[day, hg] for hg in model.HG)
        vars += sum(model.RC_FG_OT_VAR[day, fg] for fg in model.FG)
        vars += sum(model.RC_DG_OT_VAR[day, dg] for dg in model.DG)
        vars += sum(model.RC_HG_OT_VAR[day, hg] for hg in model.HG)
        demand = vars*hr_time_cost.loc[str(day), jg]
        left_side += model.RC_HG_BASE_VAR[day, jg]
        left_side += model.RC_HG_OT_VAR[day, jg]
    return left_side >= demand
model.add_component('DAILY_DEMAND_CONST', pyo.Constraint(model.DAYS, model.TG, rule=daily_demand_rule))
# ----------------------------------------------------------------

In [49]:
# ------------------------- SOLVE ---------------------------
solver = pyo.SolverFactory('glpk')
result = solver.solve(model, tee=True, keepfiles=False)

GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 --write /var/folders/xd/s2w980l53933j4xkhzvkjbmm0000gn/T/tmpdvt2_xjg.glpk.raw
 --wglp /var/folders/xd/s2w980l53933j4xkhzvkjbmm0000gn/T/tmprzv5wd50.glpk.glp
 --cpxlp /var/folders/xd/s2w980l53933j4xkhzvkjbmm0000gn/T/tmpflybbrkh.pyomo.lp
Reading problem data from '/var/folders/xd/s2w980l53933j4xkhzvkjbmm0000gn/T/tmpflybbrkh.pyomo.lp'...
7658 rows, 13765 columns, 109595 non-zeros
160105 lines were read
Writing problem data to '/var/folders/xd/s2w980l53933j4xkhzvkjbmm0000gn/T/tmprzv5wd50.glpk.glp'...
150667 lines were written
GLPK Simplex Optimizer 5.0
7658 rows, 13765 columns, 109595 non-zeros
Preprocessing...
670 rows, 7457 columns, 56070 non-zeros
Scaling...
 A: min|aij| =  3.408e-03  max|aij| =  1.000e+00  ratio =  2.934e+02
GM: min|aij| =  5.848e-02  max|aij| =  1.710e+01  ratio =  2.924e+02
EQ: min|aij| =  3.420e-03  max|aij| =  1.000e+00  ratio =  2.924e+02
Constructing initial basis...
Size of triangular par

In [51]:
fg_base = model.CE_BASE_VAR.extract_values()
fg_ot = model.CE_OT_VAR.extract_values()
opt = {day:{dg:pyo.value(model.CE_BASE_VAR[day, dg]) for dg in model.CEMP} for day in model.DAYS}
opt2 = {day:{dg:pyo.value(model.CE_OT_VAR[day, dg]) for dg in model.CEMP} for day in model.DAYS}

# new food category
df = pd.DataFrame(opt).transpose().sum(axis=1)
df2 = pd.DataFrame(opt2).transpose().sum(axis=1)
old_df = overtime_hours.transpose().groupby(employees['job_title']).sum().loc[food_related].transpose().sum(axis=1)
old_df2 = total_hours.transpose().groupby(employees['job_title']).sum().loc[food_related].transpose().sum(axis=1)
# plot the data
fig = go.Figure()
# for i in df.columns:
fig.add_trace(go.Scatter(x=df.index, y=df, opacity=1, name='Optimized'))
fig.add_trace(go.Scatter(x=df2.index, y=df2, opacity=1, name='Optimized OT'))
# for i in old_df.columns:
fig.add_trace(go.Scatter(x=old_df.index, y=old_df, opacity=0.4, name='Old'))
fig.update_layout(title='FOOD', xaxis_title='Date', yaxis_title='Minutes')
fig.show()

fig2 = go.Figure()
fig2.add_trace(go.Scatter(x=df.index, y=df+df2, opacity=1, name='Optimized Total'))
fig2.add_trace(go.Scatter(x=old_df.index, y=old_df2, opacity=0.4, name='Old total'))
fig2.update_layout(title='FOOD', xaxis_title='Date', yaxis_title='Minutes')
fig2.show()

In [52]:
data = total_hours.transpose().groupby(employees['job_title']).sum().loc[hr_related].transpose()
fig = go.Figure()
for i in data.columns:
    fig.add_trace(go.Scatter(x=data.index, y=data[i], name=i))
fig.update_layout(title='FOOD', xaxis_title='Date', yaxis_title='Minutes')
fig.show()
